In [ ]:
pip install gdown

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon
from tqdm import tqdm
import gdown
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)
import plotly.express as px

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
file_id = "1U34r_SnEZNpIPXM-_lRDjlin3vbTN_sL"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Download the file
gdown.download(url, "FD_SALAAN_2021.csv", quiet=False)

In [ ]:
base = pd.read_csv("FD_SALAAN_2021.csv", delimiter=";")

In [ ]:
base.head()

PICO stands for
 • Population : Who are we interested in? : all French population in 2021 (all employed French population ?)
 • Intervention : What treatment/intervention do we study? : do we find a specific treatment ? related to sex/geography/PCS? determined by lasso selection ?
 • Comparison : What are we comparing it to? depends on the treatment
 •Outcome : What are we interested in? wages 

### Treatment of the data 

In [ ]:
base.columns

In [ ]:
base.describe()

In [ ]:
numerical_columns = [
    'DATDEB', 'DATFIN', #date début et fin de rémunération par rapport au 01/01
     'AGE', #age en années
     'POND', #pondération 1/12e
    'DUREE', #durée de paie en jours
      'NBHEUR', 'NBHEUR_TOT', #nombre d'heures salariées total (quelle diff?)
]
categorical_columns = [
    'A6', 'A17', 'A38' #activité en nomenclature agrégrée
    'CPFD', #temps complet ou partiel
    'DEPR', 'DEPT', #département résidence et travail
    'DOMEMPL', 'DOMEMPL_EM', #domaine de l'emploi et l'établissement d'affectation/employeur
    
    'FILT', #indic poste annexe 2 ou non-annexe 1 (seuils rémunération volume)
    'REGR', 'REGT', #région de résidence et de travail
    'SEXE', #1 homme 2 femme
    'PCS', #PCS-ESE
    'TYP_EMPLOI', #ordinaire, apprenti, autre
    'CONV_COLL', #convention collective
  
    'TRNNETO', #rémunération nette globale en tranches -> à passer en numérique ?
    'TRALCHT', #total des indémnités de chômage, en tranches -> passage en numérique ?
    'TREFF', #tranche d'effectif : de 0 à 250+ postes
    'CONT_TRAV', #contrat de travail : APP apprentissage, TOA occasionnel ou à l'acte, TTP intérim, AUTre
    'CS', #CSP mais code plus simple 
    'AGE_TR', #age en tranches quadriennales
    'DATDEB_TR',
       'DATFIN_TR', #dates début et fin rémunération en tranches
    'DUREE_TR', #durée de paie exprimée en jours en tranches mensuelles
]

### Changing the names of some categorical variables

In [ ]:
#domaine d'emploi

code_to_name = {
    1 : "Fonction Publique d'Etat",
    2 : 'Fonction Publique Territoriale',
    3 : 'Fonction Publique Hospitalière',
    4 : 'Autres Organismes Publics Administratifs',
    5 : "Personnes morales de droit public soumises au droit commercial",
6 : "Entreprises individuelles",
7 : "Particuliers Employeurs", 
8 : "Organismes privés spécialisés et groupements de droit privé",
9 : "Autres sociétés privées"
}

base['DOMEMPL_EM_N'] = base['DOMEMPL_EM'].replace(code_to_name)

base['DOMEMPL_N'] = base['DOMEMPL'].replace(code_to_name)

In [ ]:
code_to_name_reg = {
    1 : "Guadeloupe",
2 :"Martinique",
3 : "Guyane",
4 : "Réunion", 
11 : "Île-de-France",
24 : "Centre-Val de Loire",
27 : "Bourgogne – Franche-Comté",
28 : "Normandie",
32 : "Nord – Pas-de-Calais – Picardie",
44 : "Alsace – Champagne-Ardenne - Lorraine",
52 : "Pays de la Loire",
53 : "Bretagne",
75 : "Aquitaine – Limousin – Poitou-Charentes",
76 : "Languedoc-Roussillon – Midi-Pyrénées",
84 : "Auvergne – Rhône-Alpes",
93 : "Provence-Alpes-Côte d'Azur",
94 : "Corse",
99 : "Étranger et TOM"}

base['REGR_N'] = base['REGR'].replace(code_to_name_reg)
base['REGT_N'] = base['REGT'].replace(code_to_name_reg)

In [ ]:
code_to_name_CS = {
   10 : "Agriculteurs (salariés de leur exploitation)",
21 : "Artisans (salariés de leur entreprise)",
22 : "Commerçants et assimilés (salariés de leur entreprise)",
23 : "Chefs d'entreprise de 10 salariés ou plus (salariés de leur entreprise)",
31 : "Professions libérales (exercées sous statut de salarié)",
33 : "Cadres de la fonction publique",
34 : "Professeurs, professions scientifiques",
35 : "Professions de l'information, des arts et des spectacles",
37 : "Cadres administratifs et commerciaux d'entreprises",
38 : "Ingénieurs et cadres techniques d'entreprises",
42 : "Professeurs des écoles, instituteurs et professions assimilées",
43 : "Professions intermédiaires de la santé et du travail social",
44 : "Clergé, religieux",
45 : "Professions intermédiaires administratives de la fonction publique",
46 : "Professions intermédiaires administratives et commerciales des entreprises",
47 : "Techniciens (sauf techniciens tertiaires)",
48 : "Contremaîtres, agents de maîtrise (maîtrise administrative exclue)",
52 : "Employés civils et agents de service de la fonction publique",
53 : "Agents de surveillance",
54 : "Employés administratifs d'entreprise",
55 : "Employés de commerce",
56 : "Personnels des services directs aux particuliers",
62 : "Ouvriers qualifiés de type industriel",
63 : "Ouvriers qualifiés de type artisanal",
64 : "Chauffeurs",
65 : "Ouvriers qualifiés de la manutention, du magasinage et du transport",
67 : "Ouvriers non qualifiés de type industriel",
68 : "Ouvriers non qualifiés de type artisanal",
69 : "Ouvriers agricoles et assimilés"}

base['CS_N'] = base['CS'].replace(code_to_name_CS)

In [ ]:
base['PCS'].describe()

In [ ]:
base['DEPR_N'].value_counts()

# TODO: change dept in int before replacing

In [ ]:
code_to_name_dep = {
    1 : "Ain",
2 : "Aisne",
3 : "Allier",
4 : "Alpes-de-Haute-Provence",
5 : "Hautes-Alpes",
6 : "Alpes-Maritimes",
7 : "Ardèche",
8 : "Ardennes",
9 : "Ariège",
10 : "Aube",
11 : "Aude",
12 : "Aveyron",
13 : "Bouches-du-Rhône",
14 : "Calvados",
15 : "Cantal",
16 : "Charente",
17 : "Charente-Maritime",
18 : "Cher",
19 : "Corrèze",
"2A" : "Corse-du-Sud",
"2B" : "Haute-Corse",
21 : "Côte-d'Or",
22 : "Côtes-d'Armor",
23 : "Creuse",
24 : "Dordogne",
25 : "Doubs",
26 : "Drôme",
27 : "Eure",
28 : "Eure-et-Loir",
29 : "Finistère",
30 : "Gard",
31 : "Haute-Garonne",
32 :"Gers",
33 : "Gironde",
34 : "Hérault",
35 : "Ille-et-Vilaine",
36 : "Indre",
37 : "Indre-et-Loire",
38 : "Isère",
39 : "Jura",
40 : "Landes",
41 : "Loir-et-Cher",
42 : "Loire",
43 : "Haute-Loire",
44 : "Loire-Atlantique",
45 : "Loiret",
46 : "Lot",
47 : "Lot-et-Garonne",
48 : "Lozère",
49 : "Maine-et-Loire",
50 : "Manche",
51 : "Marne",
52 : "Haute-Marne",
53 : "Mayenne",
54 : "Meurthe-et-Moselle",
55 : "Meuse",
56 : "Morbihan",
57 : "Moselle",
58 : "Nièvre",
59 : "Nord",
60 : "Oise",
61 : "Orne",
62 : "Pas-de-Calais",
63 : "Puy-de-Dôme",
64 : "Pyrénées-Atlantiques",
65 : "Hautes-Pyrénées",
66 : "Pyrénées-Orientales",
67 : "Bas-Rhin",
68 : "Haut-Rhin",
69 : "Rhône",
70 : "Haute-Saône",
71 : "Saône-et-Loire",
72 : "Sarthe",
73 : "Savoie",
74 : "Haute-Savoie",
75 : "Paris",
76 : "Seine-Maritime",
77 : "Seine-et-Marne",
78 : "Yvelines",
79 : "Deux-Sèvres",
80 : "Somme",
81 : "Tarn",
82 : "Tarn-et-Garonne",
83 : "Var",
84 : "Vaucluse",
85 : "Vendée",
86 : "Vienne",
87 : "Haute-Vienne",
88 : "Vosges",
89 : "Yonne",
90 : "Territoire de Belfort",
91 : "Essonne",
92 : "Hauts-de-Seine",
93 : "Seine-Saint-Denis",
94 : "Val-de-Marne",
95 : "Val-d'Oise",
971 : "Guadeloupe",
972 : "Martinique",
973 : "Guyane",
974 : "La Réunion",
99 : "Etranger et TOM"}

base['DEPR_N'] = base['DEPR'].replace(code_to_name_dep)
base['DEPT_N'] = base['DEPT'].replace(code_to_name_dep)

In [ ]:
base.dtypes

### exporting the dataframe

In [ ]:
base.to_csv('INSEE_DATA_TREATED.csv')